In [4]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [35]:
#計測地点のosm_idを指定
start_osm = 1009201303
mid_osm = [1009201310,313277121]
end_osm =  152781248
start_mesh = 50326611424
end_mesh = 50326612314

In [36]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [37]:
getdbname = "kddi_12_new_param"
dbname = "kddi_trkn12_monthly_anbun"
trknName = "trkn_12_grp"
sttime = "2022-04-01 12:00:00"

In [38]:
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                cur.execute(sql)
                connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        cur.execute(sql)
        connection.commit()
        cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)


33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値61.81666666666667距離0.0 km
50326612313
存在メッシュ：50326612313値67.85833333350595距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値67.85833333350595距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値61.81666666666667距離0.0 km
50326612313
存在メッシュ：50326612313値67.85833333350595距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値73.9距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値73.9距離0.2892176975888725 km
33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値66.9距離0.0 km
50326612313
存在メッシュ：50326612313値76.16666666693143距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値76.16666666693143距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値66.9距離0.0 km
50326612313
存在メッシュ：50326612313値76.16666666693143距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値85.43333333333334距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値85.43333333333334距離0.289217

33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値69.33333333333333距離0.0 km
50326612313
存在メッシュ：50326612313値79.13333333361334距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値79.13333333361334距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値69.33333333333333距離0.0 km
50326612313
存在メッシュ：50326612313値79.13333333361334距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値88.93333333333332距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値88.93333333333332距離0.2892176975888725 km
33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値71.15距離0.0 km
50326612313
存在メッシュ：50326612313値79.30833333356644距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値79.30833333356644距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値71.15距離0.0 km
50326612313
存在メッシュ：50326612313値79.30833333356644距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値87.46666666666667距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314

33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値62.61666666666667距離0.0 km
50326612313
存在メッシュ：50326612313値68.94166666684738距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値68.94166666684738距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値62.61666666666667距離0.0 km
50326612313
存在メッシュ：50326612313値68.94166666684738距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値75.26666666666667距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値75.26666666666667距離0.2892176975888725 km
33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値66.55距離0.0 km
50326612313
存在メッシュ：50326612313値74.20833333355215距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値74.20833333355215距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値66.55距離0.0 km
50326612313
存在メッシュ：50326612313値74.20833333355215距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値81.86666666666666距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314

33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値61.81666666666667距離0.0 km
50326612313
存在メッシュ：50326612313値67.85833333350595距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値67.85833333350595距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値61.81666666666667距離0.0 km
50326612313
存在メッシュ：50326612313値67.85833333350595距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値73.9距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値73.9距離0.2892176975888725 km
33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値66.9距離0.0 km
50326612313
存在メッシュ：50326612313値76.16666666693143距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値76.16666666693143距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値66.9距離0.0 km
50326612313
存在メッシュ：50326612313値76.16666666693143距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値85.43333333333334距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値85.43333333333334距離0.289217

33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値69.33333333333333距離0.0 km
50326612313
存在メッシュ：50326612313値79.13333333361334距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値79.13333333361334距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値69.33333333333333距離0.0 km
50326612313
存在メッシュ：50326612313値79.13333333361334距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値88.93333333333332距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値88.93333333333332距離0.2892176975888725 km
33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値71.15距離0.0 km
50326612313
存在メッシュ：50326612313値79.30833333356644距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値79.30833333356644距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値71.15距離0.0 km
50326612313
存在メッシュ：50326612313値79.30833333356644距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値87.46666666666667距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314

...

33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値69.33333333333333距離0.0 km
50326612313
存在メッシュ：50326612313値79.13333333361334距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値79.13333333361334距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値69.33333333333333距離0.0 km
50326612313
存在メッシュ：50326612313値79.13333333361334距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値88.93333333333332距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値88.93333333333332距離0.2892176975888725 km
33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値71.15距離0.0 km
50326612313
存在メッシュ：50326612313値79.30833333356644距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値79.30833333356644距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値71.15距離0.0 km
50326612313
存在メッシュ：50326612313値79.30833333356644距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値87.46666666666667距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314

33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値62.61666666666667距離0.0 km
50326612313
存在メッシュ：50326612313値68.94166666684738距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値68.94166666684738距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値62.61666666666667距離0.0 km
50326612313
存在メッシュ：50326612313値68.94166666684738距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値75.26666666666667距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値75.26666666666667距離0.2892176975888725 km
33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値66.55距離0.0 km
50326612313
存在メッシュ：50326612313値74.20833333355215距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値74.20833333355215距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値66.55距離0.0 km
50326612313
存在メッシュ：50326612313値74.20833333355215距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値81.86666666666666距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314

33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値61.81666666666667距離0.0 km
50326612313
存在メッシュ：50326612313値67.85833333350595距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値67.85833333350595距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値61.81666666666667距離0.0 km
50326612313
存在メッシュ：50326612313値67.85833333350595距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値73.9距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値73.9距離0.2892176975888725 km
33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値66.9距離0.0 km
50326612313
存在メッシュ：50326612313値76.16666666693143距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値76.16666666693143距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値66.9距離0.0 km
50326612313
存在メッシュ：50326612313値76.16666666693143距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値85.43333333333334距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値85.43333333333334距離0.289217

33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値69.33333333333333距離0.0 km
50326612313
存在メッシュ：50326612313値79.13333333361334距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値79.13333333361334距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値69.33333333333333距離0.0 km
50326612313
存在メッシュ：50326612313値79.13333333361334距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値88.93333333333332距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値88.93333333333332距離0.2892176975888725 km
33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値71.15距離0.0 km
50326612313
存在メッシュ：50326612313値79.30833333356644距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値79.30833333356644距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値71.15距離0.0 km
50326612313
存在メッシュ：50326612313値79.30833333356644距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値87.46666666666667距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314

33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値62.61666666666667距離0.0 km
50326612313
存在メッシュ：50326612313値68.94166666684738距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値68.94166666684738距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値62.61666666666667距離0.0 km
50326612313
存在メッシュ：50326612313値68.94166666684738距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値75.26666666666667距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314値75.26666666666667距離0.2892176975888725 km
33.8473958335:33.8473958335:132.774218750:132.774218750
50326611424
存在メッシュ：50326611424値66.55距離0.0 km
50326612313
存在メッシュ：50326612313値74.20833333355215距離0.14460884879856797 km
50326612313
存在メッシュ：50326612313値74.20833333355215距離0.14460884879856797 km
50326611424
存在メッシュ：50326611424値66.55距離0.0 km
50326612313
存在メッシュ：50326612313値74.20833333355215距離0.14460884879856797 km
50326612314
存在メッシュ：50326612314値81.86666666666666距離0.2892176975888725 km
50326612314
存在メッシュ：50326612314

In [39]:
connection.close()

In [ ]:
#CreateGeojsonFile